<a href="https://colab.research.google.com/github/Songjaeheon0923/RE-ALthon_2024/blob/main/model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



# **Get the code and models**

In [1]:
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1


from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output
clear_output()
print("\nDone")


Done




---



# **Quick guide**
1. Create video file named input_video.mp4 - audio track removed
2. Create audio file named input_audio.wav
3. Both files have to be exact same length
4. Target face in the input_video.mp4, must be "detectable" in ALL videoframes (So no black or blurry frames etc)
5. Make sure u use correct file extensions
6. wav2lip does not like very long and high res clips (1080p/30seconds max)



# 생성할 사람 사진, 및 text이용하여 영상 생성

In [2]:
!pip install gtts
!pip install pydub
!apt-get install -y ffmpeg  # ffmpeg 설치

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
from gtts import gTTS
import os

def generate_tts(text, output_path):
    tts = gTTS(text=text, lang='ko')  # 한국어 설정
    tts.save(output_path)
    print(f"TTS 파일이 생성되었습니다: {output_path}")

    os.system(f"start {output_path}" if os.name == "nt" else f"open {output_path}")

text = "고려 대학교"  # 변환할 텍스트
output_path = "/content/sample_data/input_audio.mp3"  # 저장할 파일 경로
generate_tts(text, output_path)

TTS 파일이 생성되었습니다: /content/sample_data/input_audio.mp3


In [4]:
from pydub import AudioSegment

def get_audio_length(file_path):
    audio = AudioSegment.from_file(file_path)
    length_in_seconds = len(audio) / 1000  # 길이를 밀리초에서 초로 변환
    return length_in_seconds

# 사용 예시
file_path = "/content/sample_data/input_audio.mp3"  # 확인할 MP3 파일 경로
length = get_audio_length(file_path)
print(f"MP3 파일의 길이: {length:.2f}초")

MP3 파일의 길이: 1.49초


In [6]:
import cv2

def create_video_from_image(image_path, output_video_path, video_length, fps):
    """
    주어진 사진을 반복하여 특정 길이의 영상으로 만드는 함수.

    :param image_path: 입력 사진 파일 경로
    :param output_video_path: 출력 영상 파일 경로
    :param video_length: 영상 길이 (초 단위)
    :param fps: 초당 프레임 수
    """
    # 이미지 읽기
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"이미지를 찾을 수 없습니다: {image_path}")

    # 이미지 크기 가져오기
    height, width, _ = image.shape

    # 동영상 코덱 설정 (MP4V 사용)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # 동영상 작성기 초기화
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 필요한 총 프레임 수 계산
    total_frames = int(video_length * fps)

    # 영상에 이미지 반복 작성
    for _ in range(total_frames):
        video_writer.write(image)

    # 동영상 작성기 종료
    video_writer.release()
    print(f"영상이 저장되었습니다: {output_video_path}")

# 사용 예시
image_path = "/content/sample_data/ai_human.jpg"  # 입력 이미지 경로
output_video_path = "/content/sample_data/input_video.mp4"  # 출력 영상 경로
video_length = length  # 영상 길이 (초)
fps = 10  # 초당 프레임 수
create_video_from_image(image_path, output_video_path, video_length, fps)

영상이 저장되었습니다: /content/sample_data/input_video.mp4


# 최종 영상 생성

In [7]:
#@title 2.Create Wav2Lip video (using wav2lip_gan.pth) GAN
!cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "/content/sample_data/input_video.mp4" --audio "/content/sample_data/input_audio.mp3"

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 14
Extracting raw audio...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libs

In [8]:
#@title 3.Play result video -  50% scaling
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/Wav2Lip/results/result_voice.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width="50%" height="50%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

In [9]:
#@title 4.Download Result.mp4 to your computer
from google.colab import files
files.download('/content/Wav2Lip/results/result_voice.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#@title 5. Delete old uploaded samples & result files, so you can start over again.
%rm /content/sample_data/*
%rm /content/Wav2Lip/results/*
from IPython.display import clear_output
clear_output()
print("\nDone! now press X")


Done! now press X
